### Implement a feed forward net on the training set

In [11]:
using Mocha, HDF5, MLBase

#1. Load the trained network weights
#2. Test the network on a random selection training set

backend = CPUBackend()
init(backend)

image_x = 32
image_y = 32
image_depth = 3

# How many samples to run in the validation test
N = 2000

#Create a memory data layer in the prediction model that we computation
#sequentially tie to the next image we want to predict on.
mem_data = MemoryDataLayer(name="data", tops=[:data], batch_size=1,
    data=Array[zeros(Float32, image_x, image_y, image_depth, 1)])

#Copy over the rest of the layers from the training network
conv1_layer = ConvolutionLayer(name="conv1", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.0001),
    bottoms=[:data], tops=[:conv1])
pool1_layer = PoolingLayer(name="pool1", kernel=(3,3), stride=(2,2), neuron=Neurons.ReLU(),
    bottoms=[:conv1], tops=[:pool1])
norm1_layer = LRNLayer(name="norm1", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool1], tops=[:norm1])
conv2_layer = ConvolutionLayer(name="conv2", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm1], tops=[:conv2], neuron=Neurons.ReLU())
pool2_layer = PoolingLayer(name="pool2", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv2], tops=[:pool2])
norm2_layer = LRNLayer(name="norm2", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool2], tops=[:norm2])
conv3_layer = ConvolutionLayer(name="conv3", n_filter=64, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm2], tops=[:conv3], neuron=Neurons.ReLU())
pool3_layer = PoolingLayer(name="pool3", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv3], tops=[:pool3])
ip1_layer   = InnerProductLayer(name="ip1", output_dim=10, weight_init=GaussianInitializer(std=0.01),
    weight_regu=L2Regu(250), bottoms=[:pool3], tops=[:ip1])

#Instead of a SoftmaxLossLayer like the training script in this file we
#have a pure SoftmaxLayer in order to determine the probability of the 
#image compared against the different n classification options
softmax_layer = SoftmaxLayer(name="prob", tops=[:prob], bottoms=[:ip1])

#Build the network
common_layers = [conv1_layer, pool1_layer, norm1_layer, conv2_layer, pool2_layer, norm2_layer,
                 conv3_layer, pool3_layer, ip1_layer]
run_net = Net("imagenet", backend, [mem_data, common_layers..., softmax_layer])

#Load the network snapshot from the training data
load_snapshot(run_net, "snapshots/snapshot-070000.jld")

#Now run 1000 training data samples through the network
fid = h5open("data/train.hdf5", "r")
features = read(fid, "data")
labels = read(fid, "label")

#Randomize the data
rp = randperm(size(features,4))
features = features[:,:,:,rp]
labels = labels[rp]

#Set up some constants used in the loop
offset = 1  #my tags vector is 1-indexed but the labels vector is 
                  #is 0-indexed.  So I have to add an offset of 1 to the label value to 
                  # match my predictive value 
println("Loading Tags...")
tags = []
fname = "batches.meta.txt"
open(fname) do file
    for line in eachline(file)
        line = strip(line)
        if length(line)>0 
            push!(tags, strip(line))
        end
    end
end

#set up a list to capture results
gts = []
preds = []

printed_flag = 0;
for j in 1:N
    if j%100==0
        println("     Running iteration $j ...") 
    end
 
    #Load the jth image and label 
    get_layer(run_net, "data").data[1][:,:,image_depth,1] = features[:,:,image_depth,j]
    gt = Int64(labels[j])+offset
    push!(gts, gt)

    # Run the net forward and use MLBase to classify the output blob
    forward(run_net)
    prediction = classify(run_net.output_blobs[:prob].data)
    push!(preds, prediction[1])
        
end

gts = convert(Array{Int64}, gts)
preds = convert(Array{Int64}, preds)
println(correctrate(gts, preds))
confusmat(10, gts, preds)

close(fid)
destroy(run_net)

20-Oct 18:28:51:INFO:root:Constructing net imagenet on Mocha.CPUBackend...
20-Oct 18:28:51:INFO:root:Topological sorting 11 layers...
20-Oct 18:28:51:INFO:root:Setup layers...
20-Oct 18:28:51:INFO:root:Network constructed!
20-Oct 18:28:51:INFO:root:Loading existing model from snapshots/snapshot-070000.jld
20-Oct 18:28:51:DEBUG:root:Loading parameters for layer conv1
20-Oct 18:28:51:DEBUG:root:Loading parameters for layer conv2
20-Oct 18:28:51:DEBUG:root:Loading parameters for layer conv3
20-Oct 18:28:51:DEBUG:root:Loading parameters for layer ip1
Loading Tags...
     Running iteration 100 ...
     Running iteration 200 ...
     Running iteration 300 ...
     Running iteration 400 ...
     Running iteration 500 ...
     Running iteration 600 ...
     Running iteration 700 ...
     Running iteration 800 ...
     Running iteration 900 ...
     Running iteration 1000 ...
     Running iteration 1100 ...
     Running iteration 1200 ...
     Running iteration 1300 ...
     Running iteration 1

### Run a validation set on the training data

In [12]:
using Mocha, HDF5

data_tr_layer = HDF5DataLayer(name="data-train", source="data/train.txt", batch_size=100, shuffle=@windows ? false : true)

data_tt_layer = HDF5DataLayer(name="data-test", source="data/test.txt", batch_size=100, shuffle=@windows ? false : true)


conv1_layer = ConvolutionLayer(name="conv1", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.0001),
    bottoms=[:data], tops=[:conv1])
pool1_layer = PoolingLayer(name="pool1", kernel=(3,3), stride=(2,2), neuron=Neurons.ReLU(),
    bottoms=[:conv1], tops=[:pool1])
norm1_layer = LRNLayer(name="norm1", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool1], tops=[:norm1])
conv2_layer = ConvolutionLayer(name="conv2", n_filter=32, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm1], tops=[:conv2], neuron=Neurons.ReLU())
pool2_layer = PoolingLayer(name="pool2", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv2], tops=[:pool2])
norm2_layer = LRNLayer(name="norm2", kernel=3, scale=5e-5, power=0.75, mode=LRNMode.WithinChannel(),
    bottoms=[:pool2], tops=[:norm2])
conv3_layer = ConvolutionLayer(name="conv3", n_filter=64, kernel=(5,5), pad=(2,2),
    stride=(1,1), filter_init=GaussianInitializer(std=0.01),
    bottoms=[:norm2], tops=[:conv3], neuron=Neurons.ReLU())
pool3_layer = PoolingLayer(name="pool3", kernel=(3,3), stride=(2,2), pooling=Pooling.Mean(),
    bottoms=[:conv3], tops=[:pool3])
ip1_layer   = InnerProductLayer(name="ip1", output_dim=10, weight_init=GaussianInitializer(std=0.01),
    weight_regu=L2Regu(250), bottoms=[:pool3], tops=[:ip1])

loss_layer  = SoftmaxLossLayer(name="softmax", bottoms=[:ip1, :label])
acc_layer   = AccuracyLayer(name="accuracy", bottoms=[:ip1, :label])

common_layers = [conv1_layer, pool1_layer, norm1_layer, conv2_layer, pool2_layer, norm2_layer,
                 conv3_layer, pool3_layer, ip1_layer]

backend = DefaultBackend()
init(backend)

# show performance on test data 
test_net = Net("CIFAR10-test", backend, [data_tt_layer, common_layers..., acc_layer])
ValidationPerformance(test_net)



20-Oct 18:29:55:INFO:root:Constructing net CIFAR10-test on Mocha.CPUBackend...
20-Oct 18:29:55:INFO:root:Topological sorting 11 layers...
20-Oct 18:29:55:INFO:root:Setup layers...
20-Oct 18:29:56:INFO:root:Network constructed!


Mocha.ValidationPerformance(************************************************************
          NAME: CIFAR10-test
       BACKEND: Mocha.CPUBackend
  ARCHITECTURE: 11 layers
............................................................
 *** Mocha.HDF5DataLayer(data-test)
    Outputs ---------------------------
          data: Blob(32 x 32 x 3 x 100)
         label: Blob(1 x 100)
............................................................
 *** Mocha.ConvolutionLayer(conv1)
    Inputs ----------------------------
          data: Blob(32 x 32 x 3 x 100)
    Outputs ---------------------------
         conv1: Blob(32 x 32 x 32 x 100)
............................................................
 *** Mocha.PoolingLayer(pool1)
    Inputs ----------------------------
         conv1: Blob(32 x 32 x 32 x 100)
    Outputs ---------------------------
         pool1: Blob(16 x 16 x 32 x 100)
............................................................
 *** Mocha.LRNLayer(norm1)
    Inputs ------